In [1]:
import matplotlib.pyplot as plt
import os
from os.path import join
import numpy as np
from PIL import Image
import matplotlib.image as mpimg
import matplotlib.image as plt
from skimage.color import rgb2gray
from skimage.color import label2rgb
from skimage.filters import gaussian
from sklearn.cluster import KMeans
import matplotlib.pyplot as patches
import matplotlib.image as pyimg

In [2]:

#plt.close('all')
clear = lambda: os.system('clear')
clear()
np.random.seed(110)

colors = [[1,0,0],[0,1,0],[0,0,1],[0,0.5,0.5],[0.5,0,0.5]]

imgNames = ['water_coins','jump','tiger']#{'balloons', 'mountains', 'nature', 'ocean', 'polarlights'};
segmentCounts = [2,3,4,5]

answer=[]
for imgName in imgNames:
    temp = 1
    for SegCount in segmentCounts:
        # Load the imageusing OpenCV        
        ###img = """ Read Image using mplib library-- 2 points """ 
        inputPath = join(''.join(['Input/', str(imgName), '.png']))
        img = mpimg.imread(inputPath)
        print('Using Matplotlib Image Library: Image is of datatype ',img.dtype,'and size ',img.shape) # Image is of type float 
        # Load the Pillow-- the Python Imaging Library
        ###img = """ Read Image using PILLOW-- 3 points""" 
        img = Image.open(inputPath) 
        img = np.asanyarray(img)
        print('Using Pillow (Python Image Library): Image is of datatype ',img.dtype,'and size ',img.shape) # Image is of type uint8  
                
        #%% %Define Parameters
        nSegments = SegCount   # of color clusters in image
        width, height = Image.open(inputPath).size
        ###nPixels = """ Compute number of image pixels from image dimensions-- 2 points""";    # Image can be represented by a matrix of size nPixels*nColors
        nPixels = width*height
        
        
        img = Image.open(inputPath) #keede
        #img = Image.fromarray(img,'RGB')
        maxIterations = 20; #maximum number of iterations allowed for EM algorithm.
        nColors = 3;
        #%% Determine the output path for writing images to files
        outputPath = join(''.join(['Output/',str(SegCount), '_segments/', imgName, '/']));
        if not(os.path.exists(outputPath)):
            os.makedirs(outputPath)
        ###""" save input image as *0.png* under outputPath-- 3 points""" #save using Matplotlib image library
        img.save( outputPath+'0.png', format='png' )
        
        
        #%% Vectorizing image for easier loops- done as im(:) in Matlab
        pixels = img
        ###pixels = """ Reshape pixels as a nPixels X nColors X 1 matrix-- 5 points"""
        pixels = np.reshape(pixels, (nPixels, nColors,1))
        
        #%%
        """ Initialize pi (mixture proportion) vector and mu matrix (containing means of each distribution)
            Vector of probabilities for segments... 1 value for each segment.
            Best to think of it like this...
            When the image was generated, color was determined for each pixel by selecting
            a value from one of "n" normal distributions. Each value in this vector 
            corresponds to the probability that a given normal distribution was chosen."""
        
        
        """ Initial guess for pi's is 1/nSegments. Small amount of noise added to slightly perturb 
           GMM coefficients from the initial guess"""
           
        pi = 1/nSegments*(np.ones((nSegments, 1),dtype='float'))
        increment = np.random.normal(0,.0001,1)
        for seg_ctr in range(len(pi)):
            if(seg_ctr%2==1):
                pi[seg_ctr] = pi[seg_ctr] + increment
            else:
                pi[seg_ctr] = pi[seg_ctr] - increment
       
    
        #%% 
        """Similarly, the initial guess for the segment color means would be a perturbed version of [mu_R, mu_G, mu_B],
           where mu_R, mu_G, mu_B respectively denote the means of the R,G,B color channels in the image.
           mu is a nSegments X nColors matrrix,(seglabels*255).np.asarray(int) where each matrix row denotes mean RGB color for a particcular segment"""
           
        ###mu = """Initialize mu to 1/nSegments*['ones' matrix (whose elements are all 1) of size nSegments X nColors] -- 5 points"""
        mu = []
        for i in range(nSegments):
            row = []
            for j in range(nColors):
                row.append(1/nSegments)
            mu.append(row)
        #print(mu)
        mu = np.asarray(mu)
        #mu.shape
        #u = i/nSegments*mu
        
        #for even start
        #add noise to the initialization (but keep it unit)
        for seg_ctr in range(nSegments):
            if(seg_ctr%2==1):
                increment = np.random.normal(0,.0001,1)
            for col_ctr in range(nColors):
                 if(seg_ctr%2==1):
                    mu[seg_ctr][col_ctr] = np.mean(pixels[:,col_ctr]) + increment
                 else:
                    mu[seg_ctr][col_ctr] = np.mean(pixels[:,col_ctr]) - increment;              
        

        
        #%% EM-iterations begin here. Start with the initial (pi, mu) guesses        
        
        mu_last_iter = mu;
        pi_last_iter = pi;
        
        
        for iteration in range(maxIterations):
            """%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
               % -----------------   E-step  -----estimating likelihoods and membership weights (Ws)
               %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"""

            print(''.join(['Image: ',imgName,' nSegments: ',str(nSegments),' iteration: ',str(iteration+1), ' E-step']))
            # Weights that describe the likelihood that pixel denoted by "pix_import scipy.miscctr" belongs to a color cluster "seg_ctr"
            Ws = np.ones((nPixels,nSegments),dtype='float')  # temporarily reinitialize all weights to 1, before they are recomputed


            """ logarithmic form of the E step."""
            
            for pix_ctr in range(nPixels):
                # Calculate Ajs
                logAjVec = np.zeros((nSegments,1),dtype='float')
                for seg_ctr in range(nSegments):
                    #sth = [np.newaxis]
                    x_minus_mu_T  = np.transpose(pixels[pix_ctr,:]-(mu[seg_ctr,:])[np.newaxis].T)
                    x_minus_mu    = ((pixels[pix_ctr,:]-(mu[seg_ctr,:])[np.newaxis].T))
                    logAjVec[seg_ctr] = np.log(pi[seg_ctr]) - .5*(np.dot(x_minus_mu_T,x_minus_mu))
                
                # Note the max
                logAmax = max(logAjVec.tolist()) 
                
                # Calculate the third term from the final eqn in the above link
                thirdTerm = 0;
                for seg_ctr in range(nSegments):
                    thirdTerm = thirdTerm + np.exp(logAjVec[seg_ctr]-logAmax)
                
                # Here Ws are the relative membership weights(p_i/sum(p_i)), but computed in a round-about way 
                for seg_ctr in range(nSegments):
                    logY = logAjVec[seg_ctr] - logAmax - np.log(thirdTerm)
                    Ws[pix_ctr][seg_ctr] = np.exp(logY)
                

            """%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                % -----------------   M-step  --------------------
               %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"""
            
            print(''.join(['Image: ',imgName,' nSegments: ',str(nSegments),' iteration: ',str(iteration+1), ' M-step: Mixture coefficients']))
            #%% temporarily reinitialize mu and pi to 0, before they are recomputed
            mu = np.zeros((nSegments,nColors),dtype='float') # mean color for each segment
            pi = np.zeros((nSegments,1),dtype='float') #mixture coefficients
            
            for seg_ctr in range(nSegments):

                denominatorSum = 0;
                for pix_ctr in range(nPixels):
                    #"""Update RGB color vector of mu[seg_ctr] as current mu[seg_ctr] + pixels[pix_ctr,:] times Ws[pix_ctr,seg_ctr] -- 5 points"""
                    #pixels2 = pixels[pix_ctr,:]
                    Ws2 = Ws.transpose()
                    result = np.multiply(Ws2[seg_ctr,pix_ctr], pixels[pix_ctr,:])
                    #mu[seg_ctr] = mu[seg_ctr] + result[1]
                    for i in range(len(mu[seg_ctr])):
                        mu[seg_ctr,i] = mu[seg_ctr, i] + result[i]
                    denominatorSum = denominatorSum + Ws[pix_ctr][seg_ctr]
                
                """Compute mu[seg_ctr] and denominatorSum directly without the 'for loop'-- 10 points.
                   If you find the replacement instruction, comment out the for loop with your solution"
                   Hint: Use functions squeeze, tile and reshape along with sum"""
                ## Update mu
                #print(11)
                mu[seg_ctr,:] =  mu[seg_ctr,:]/ denominatorSum;
                ## Update pi
                pi[seg_ctr] = denominatorSum / nPixels; #sum of weights (each weight is a probability) for given segment/total num of pixels   
        


            print(np.transpose(pi))

            muDiffSq = np.sum(np.multiply((mu - mu_last_iter),(mu - mu_last_iter)))
            piDiffSq = np.sum(np.multiply((pi - pi_last_iter),(pi - pi_last_iter)))

            if (muDiffSq < .0000001 and piDiffSq < .0000001): #sign of convergence
                print('Convergence Criteria Met at Iteration: ',iteration, '-- Exiting code')
                break;
            

            mu_last_iter = mu;
            pi_last_iter = pi;

            
            ##Draw the segmented image using the mean of the color cluster as the 
            ## RGB value for all pixels in that cluster.
            segpixels = np.array(pixels)
            cluster = 0
            for pix_ctr in range(nPixels):
                cluster = np.where(Ws[pix_ctr,:] == max(Ws[pix_ctr,:]))
                vec     = np.squeeze(np.transpose(mu[cluster,:])) 
                segpixels[pix_ctr,:] =  vec.reshape(vec.shape[0],1)
            
            """ Save segmented image at each iteration. For displaying consistent image clusters, it would be useful to blur/smoothen the segpixels image using a Gaussian filter.  
                Prior to smoothing, convert segpixels to a Grayscale image, and convert the grayscale image into clusters based on pixel intensities"""

            img = np.asanyarray(img)
            print(img.dtype)
            
            img = np.asanyarray(img)
            segpixels = np.reshape(segpixels,(img.shape[0],img.shape[1],nColors)) ## reshape segpixels to obtain R,G, B image
            #segpixels = """convert segpixels to uint8 gray scale image and convert to grayscale-- 5 points""" #convert to grayscale
            segpixels = segpixels.astype(np.uint8)##check cell above
            #print(segpixels.dtype)
            im2 = rgb2gray(segpixels)
            #plt.imshow(im2)
            #plt.show()  
            
            #from sklearn.cluster import KMeans
            ###kmeans =
            """ Use kmeans from sci-kit learn library to cluster pixels in gray scale segpixels image to *nSegments* clusters-- 10 points"""
            #x, y, z = segpixels.shape
            kmeans = KMeans(nSegments)
            #im2 = im2.flatten()
            im2 = np.reshape(im2, (im2.shape[0]*im2.shape[1], 1))
            kmeans.fit(im2)
 

            #seglabels = 
            """ reshape kmeans.labels_ output by kmeans to have the same size as segpixels -- 5 points"""
            seglabels = np.reshape(np.array(kmeans.labels_, dtype=np.uint8), (img.shape[0], img.shape[1]))
            
            ###seglabels = 
            """Use np.clip, Gaussian smoothing with sigma =2 and label2rgb functions to smoothen the seglabels image, and output a float RGB image with pixel values between [0--1]-- 20 points"""
            
            #im_blurred = gaussian_filter(im, sigma=2.5) #(5,5,1)
            #colors = [(1, 0, 0), (0, 1, 0), (0, 0, 1)]
            seglabels = label2rgb(seglabels, image = img, colors = colors, alpha = 1, image_alpha = 1)
            seglabels = gaussian(seglabels, sigma=2)
            seglabels = np.clip(seglabels, 0, 1)
            
            mpimg.imsave(''.join([outputPath,str(iteration+1),'.png']),seglabels) #save the segmented output
            
            """ Display the 20th iteration (or final output in case of convergence) segmentation images with nSegments = 2,3,4,5 for the three images-- this will be a 3 row X 4 column image matrix-- 15 points"""  
            temp = seglabels
        answer.append(temp)
        
print(len(answer))
for i in range (12):
    plt.pyplot.subplot(3,4,i+1)
    plt.imshow(answer[i])
    
plt.show()

""" Comment on the results obtained, and discuss your understanding of the Image Segmentation problem in general-- 10 points """ 
"""check readme.txt"""

Using Matplotlib Image Library: Image is of datatype  float32 and size  (312, 252, 3)
Using Pillow (Python Image Library): Image is of datatype  uint8 and size  (312, 252, 3)
Image: water_coins nSegments: 2 iteration: 1 E-step
Image: water_coins nSegments: 2 iteration: 1 M-step: Mixture coefficients
[[0.49996714 0.50003286]]
uint8
Image: water_coins nSegments: 2 iteration: 2 E-step


/home/akansha/anaconda3/lib/python3.7/site-packages/skimage/filters/_gaussian.py:122: RuntimeWarning: Images with dimensions (M, N, 3) are interpreted as 2D+RGB by default. Use `multichannel=False` to interpret as 3D image with last dimension of length 3.
  warn(RuntimeWarning(msg))


Image: water_coins nSegments: 2 iteration: 2 M-step: Mixture coefficients
[[0.44622235 0.55377765]]
uint8
Image: water_coins nSegments: 2 iteration: 3 E-step


/home/akansha/anaconda3/lib/python3.7/site-packages/skimage/filters/_gaussian.py:122: RuntimeWarning: Images with dimensions (M, N, 3) are interpreted as 2D+RGB by default. Use `multichannel=False` to interpret as 3D image with last dimension of length 3.
  warn(RuntimeWarning(msg))


Image: water_coins nSegments: 2 iteration: 3 M-step: Mixture coefficients
[[0.44233313 0.55766687]]
uint8
Image: water_coins nSegments: 2 iteration: 4 E-step


...

/home/akansha/anaconda3/lib/python3.7/site-packages/skimage/filters/_gaussian.py:122: RuntimeWarning: Images with dimensions (M, N, 3) are interpreted as 2D+RGB by default. Use `multichannel=False` to interpret as 3D image with last dimension of length 3.
  warn(RuntimeWarning(msg))


Image: tiger nSegments: 5 iteration: 20 E-step
Image: tiger nSegments: 5 iteration: 20 M-step: Mixture coefficients
[[0.32617766 0.14075987 0.28380583 0.12357967 0.12567697]]
uint8


/home/akansha/anaconda3/lib/python3.7/site-packages/skimage/filters/_gaussian.py:122: RuntimeWarning: Images with dimensions (M, N, 3) are interpreted as 2D+RGB by default. Use `multichannel=False` to interpret as 3D image with last dimension of length 3.
  warn(RuntimeWarning(msg))


12


AttributeError: module 'matplotlib.image' has no attribute 'pyplot'